# TSA Chapter 5: ARMA-GARCH Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_armagarch/TSA_ch5_armagarch.ipynb)

Python implementation of combined ARMA-GARCH model for mean and variance modeling.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
import yfinance as yf
from arch import arch_model

In [ ]:
# Download S&P 500 data
sp = yf.download('^GSPC', start='2015-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
returns = (sp['Close'].pct_change() * 100).dropna()
print(f"S&P 500 returns: {len(returns)} observations")

In [ ]:
# Fit ARMA(1,1)-GARCH(1,1) model
am = arch_model(returns, mean='ARX', lags=1, vol='Garch', p=1, q=1, dist='t')
res = am.fit(disp='off')
print(res.summary())

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
axes[0].plot(returns.index, returns.values, color=BLUE, lw=0.4, alpha=0.8)
axes[0].set_title('S&P 500 Returns', fontweight='bold')
axes[0].set_ylabel('Return (%)')

axes[1].plot(returns.index, res.conditional_volatility, color=RED, lw=0.6)
axes[1].fill_between(returns.index, 0, res.conditional_volatility, color=RED, alpha=0.2)
axes[1].set_title('ARMA(1,1)-GARCH(1,1) Conditional Volatility', fontweight='bold')
axes[1].set_ylabel('Volatility (%)')
axes[1].set_xlabel('Date')

fig.tight_layout(); plt.show()
print(f"\nAIC: {res.aic:.2f}, BIC: {res.bic:.2f}")